# Exploración Inicial

__Descripción:__

Se obtiene el AGEB de origen y destino de los viajes de los viajes de ecobici y se obtiene el agregado primedio de los viajes entre AGEBS por hora.

__Input__

- Estaciones de ecobici: Estaciones de ecobici con clave AGEB obtenida de __Uber © 2019 Copyright Uber Technologies, Inc.Data Attributions__

- Viajes Ecobici 2019
Responsable: Ecobici



In [1]:
import pandas as pd
import pickle

import pandas as pd
import dask.dataframe as dd
import time

import scipy.stats
import re

## Obtencion de AGEBS de interés:

In [2]:
df_estaciones = pd.read_csv("/Users/danielbustillos/Documents/ecobici_viz/data/SIG/procesado/CSV/estaciones-de-ecobici_con_AGEB.csv")
df_estaciones.head()

,ID,Nombre,Dirección,Número,Código po,districtCo,Colonia,altitude,nearbyStat,location/l,...,nearbySt_3,nearbySt_4,nearbySt_5,punto_geo,CVE_AGEB,CVE_MUN,CVE_LOC,CVE_ENT,MOVEMENT_I,DISPLAY_NA
0,440,440 REAL DE MAYORAZGO-PUENTE XOCO,440 - Real De Mayorazgo-Puente Xoco,S/N,3330.0,1,Ampliación Granada,NaN,433,19.360650,...,NaN,NaN,NaN,"19.36065,-99.168669",0967,14,1,9,4550,Movement Zone 4550
1,283,283 AV. COYOACÁN-RAFAEL DONDE,283 - Av. Coyoacán-Rafael Donde,S/N,3103.0,1,Ampliación Granada,NaN,279,19.398270,...,NaN,NaN,NaN,"19.39827,-99.167383",0045,14,1,9,4551,Movement Zone 4551
2,279,279 LA MORENA-PROVIDENCIA,279 - La Morena-Providencia,S/N,3103.0,DVN,Del Valle Norte,NaN,164,19.399908,...,NaN,NaN,NaN,"19.399908,-99.166553",0045,14,1,9,4551,Movement Zone 4551
3,443,443 BRUNO TRAVEN-GOLONDRINAS,443 - Bruno Trave-Avenida México-Coyoacan,S/N,3340.0,1,Ampliación Granada,NaN,432,19.359655,...,NaN,NaN,NaN,"19.359655,-99.162025",0971,14,1,9,4553,Movement Zone 4553
4,432,432 CARRILLO PUERTO-AV. MÉXICO-COYOACÁN,432 - Carrillo Pto-Golondrinas,S/N,3340.0,GPM,General Pedro Maria Anaya,NaN,442,19.361296,...,NaN,NaN,NaN,"19.361296,-99.162581",0971,14,1,9,4553,Movement Zone 4553


In [3]:
df_estaciones['CVE_AGEB'] = [''.join(filter(lambda x: x.isdigit(), row)) for row in df_estaciones['CVE_AGEB']]
df_estaciones['CVE_AGEB'] = df_estaciones['CVE_AGEB'].astype(int)

In [4]:
lista_agebs = df_estaciones['CVE_AGEB'].astype(int).unique().tolist()
len(lista_agebs)

143

In [5]:
lista_agebs[:5]

[967, 45, 971, 219, 187]

Guardamos lo AGEBS en una lista:

In [6]:
with open("../data/lista_agebs.txt", "wb") as fp:
    pickle.dump(lista_agebs, fp)

## Leamos el archivo de viajes de UBER

In [7]:
path_uber_trips = "../data/uber/mexico_city-agebs-2019-1-OnlyWeekdays-HourlyAggregate.csv"
path_uber_trips_0 = "../data/uber/mexico_city-agebs-2019-2-All-HourlyAggregate.csv"
path_uber_trips_1 = "../data/uber/mexico_city-agebs-2018-3-All-HourlyAggregate.csv"

##### Leemos los CSV de los viajes de uber y luego los unimos y por ultimos hacemos un GB

In [8]:
t = time.clock()
df_trips = dd.read_csv(path_uber_trips)
df_trips = df_trips.compute()
df_trips = df_trips[(df_trips["sourceid"].isin(lista_agebs)) & df_trips["dstid"].isin(lista_agebs)]
print("load train: " , time.clock()-t)
df_trips.head()

load train:  80.375429


,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
1755,1017,891,18,947.97,298.44,907.59,1.33
2187,767,894,10,603.12,1311.17,321.83,2.42
2925,875,872,4,624.35,400.41,527.76,1.75
5090,983,1430,23,2180.55,484.67,2136.46,1.21
5638,1375,1002,20,2882.15,747.86,2804.85,1.25


In [9]:
t = time.clock()
df_trips_0 = dd.read_csv(path_uber_trips_0)
df_trips_0 = df_trips_0.compute()
df_trips_0 = df_trips_0[(df_trips_0["sourceid"].isin(lista_agebs)) & df_trips_0["dstid"].isin(lista_agebs)]
print("load train: " , time.clock()-t)
df_trips_0.head()

load train:  117.945552


,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
3550,837,875,18,1393.31,449.05,1335.93,1.32
3567,825,995,18,1383.30,272.89,1355.85,1.22
4440,637,1002,10,1510.11,348.65,1473.44,1.24
8760,384,806,22,1952.67,546.02,1881.95,1.31
10274,1002,656,9,1502.75,317.97,1475.48,1.20


In [10]:
t = time.clock()
df_trips_1 = dd.read_csv(path_uber_trips_1)
df_trips_1 = df_trips_1.compute()
df_trips_1 = df_trips_1[(df_trips_1["sourceid"].isin(lista_agebs)) & df_trips_1["dstid"].isin(lista_agebs)]

print("load train: " , time.clock()-t)
df_trips_1.head()

load train:  107.91800200000003


,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
745,64,581,17,1918.42,430.38,1871.45,1.25
2645,1029,926,22,913.00,342.60,856.20,1.42
5023,810,767,22,1319.95,500.68,1242.42,1.40
12401,793,1093,13,1901.10,536.18,1827.43,1.33
17435,952,948,21,962.11,302.37,921.29,1.33


##### Append

In [11]:
df_trips = df_trips.append(df_trips_0) 
df_trips = df_trips.append(df_trips_1) 

In [12]:
del df_trips_0
del df_trips_1

### Gb

In [13]:
df_trips = df_trips.groupby([
    'sourceid','dstid',"hod"])[["mean_travel_time","standard_deviation_travel_time",]].mean().reset_index(drop=False)

In [14]:
df_trips.shape

(59622, 5)

In [15]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time
0,37,204,0,1422.04,249.155
1,37,204,3,2005.00,706.710
2,37,204,4,1792.55,756.240
3,37,204,5,1946.88,471.540
4,37,204,6,1893.89,556.570


In [16]:
list(set(lista_agebs) - set(df_trips["sourceid"].unique()))

[5,
 11,
 1356,
 45,
 1211,
 783,
 1360,
 49,
 401,
 533,
 1337,
 444,
 187,
 1214,
 1341,
 1182]

In [17]:
df_trips = df_trips[(df_trips["sourceid"].isin(lista_agebs)) & df_trips["dstid"].isin(lista_agebs)]

In [18]:
#df_trips.to_csv('./data/procesado/mexico_city-agebs-2019-2-All-HourlyAggregate_filter.csv')

In [19]:
df_trips.reset_index(drop=True, inplace=True)

In [20]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time
0,37,204,0,1422.04,249.155
1,37,204,3,2005.00,706.710
2,37,204,4,1792.55,756.240
3,37,204,5,1946.88,471.540
4,37,204,6,1893.89,556.570


In [21]:
df_trips.shape

(59622, 5)

### tiempo máximo y mínimos por viaje y hora.
Suponemos una distribución normal con media 'mean_travel_time' y desviación estandar 'standard_deviation_travel_time>'

In [22]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time
0,37,204,0,1422.04,249.155
1,37,204,3,2005.00,706.710
2,37,204,4,1792.55,756.240
3,37,204,5,1946.88,471.540
4,37,204,6,1893.89,556.570


In [23]:
def travel_min(mean,std):
    min = scipy.stats.norm(mean, std).ppf(.10)
    #max = scipy.stats.norm(mean, std).ppf(.90)
    return(min)
def travel_max(mean,std):
    #min = scipy.stats.norm(mean, std).ppf(.10)
    max = scipy.stats.norm(mean, std).ppf(.90)
    return(max)

In [24]:
df_trips['min_travel_time'] = 0
df_trips['max_travel_time'] = 0

In [25]:
df_trips['min_travel_time'] =  df_trips.apply(
    lambda row : travel_min(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [26]:
df_trips['max_travel_time'] =  df_trips.apply(
    lambda row : travel_max(row['mean_travel_time'], 
                            row['standard_deviation_travel_time']), axis = 1)

In [27]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,min_travel_time,max_travel_time
0,37,204,0,1422.04,249.155,1102.735020,1741.344980
1,37,204,3,2005.00,706.710,1099.314693,2910.685307
2,37,204,4,1792.55,756.240,823.389444,2761.710556
3,37,204,5,1946.88,471.540,1342.577175,2551.182825
4,37,204,6,1893.89,556.570,1180.616845,2607.163155


## Definimos periodos del día para non graficar todas las horas:

In [28]:
def periodo_hora(row):
    if row in [7,8,9]:
        return "7-10"
    elif row in[10,11,12]:
        return "10-13"
    elif row in [13,14,15]:
        return "13-16"
    elif row in [16,17,18]:
        return "16-19"
    elif row in [19,20,21]:
        return "19-22"
    elif row in [22,23,0]:
        return "22-1"
    else:
        return None

df_trips["Hora_Retiro_bin"] = df_trips["hod"].apply(periodo_hora)

df_trips = df_trips.dropna()

In [29]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,min_travel_time,max_travel_time,Hora_Retiro_bin
0,37,204,0,1422.040,249.155,1102.735020,1741.344980,22-1
5,37,204,7,1823.000,375.840,1341.341660,2304.658340,7-10
6,37,204,8,1920.180,635.330,1105.971844,2734.388156,7-10
7,37,204,9,1693.570,310.490,1295.661054,2091.478946,7-10
8,37,204,10,1722.045,393.405,1217.876206,2226.213794,10-13


In [32]:
df_trips = df_trips.groupby(["sourceid","dstid","Hora_Retiro_bin"]).mean().reset_index(drop=False)

## Convertimos la duración de los viajes de segundos a minutos:

In [33]:
df_trips['max_travel_time']  = df_trips['max_travel_time'] / 60
df_trips['min_travel_time']  = df_trips['min_travel_time'] / 60
df_trips['mean_travel_time']  = df_trips['mean_travel_time'] / 60

In [34]:
df_trips.columns

Index(['sourceid', 'dstid', 'Hora_Retiro_bin', 'hod', 'mean_travel_time',
       'standard_deviation_travel_time', 'min_travel_time', 'max_travel_time'],
      dtype='object')

seleccionamos columnnas

In [35]:
df_trips = df_trips[['sourceid', 'dstid', 'hod', 'mean_travel_time', 'min_travel_time',
       'max_travel_time', 'Hora_Retiro_bin']]

In [36]:
df_trips.head()

,sourceid,dstid,hod,mean_travel_time,min_travel_time,max_travel_time,Hora_Retiro_bin
0,37,204,11.0,29.763528,19.350815,40.176241,10-13
1,37,204,14.0,33.085083,17.868296,48.301871,13-16
2,37,204,17.0,29.529481,19.691105,39.367858,16-19
3,37,204,20.0,30.265528,16.567806,43.963249,19-22
4,37,204,15.0,26.734778,18.039344,35.430212,22-1


In [37]:
df_trips.to_csv("../data/production_data/viajes_uber.csv", index=False)

In [38]:
df_trips[df_trips["sourceid"]==]

SyntaxError: invalid syntax (<ipython-input-38-2fee4e9bf687>, line 1)